In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import threading
import time

In [2]:
def scrapping_genre(genre,ll):
    data = []
    print(genre)
    url_genre = ll
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    # response = requests.get(url)
    response_genre = requests.get(url_genre, headers=headers)
    soup_genre = BeautifulSoup(response_genre.text,'html.parser')
    # print(soup)# Prettify the HTML
    pretty_html = soup_genre.prettify()
    list_class = soup_genre.find_all('li',class_='ipc-metadata-list-summary-item')
    list_class_size = 0
    for i in list_class:
        list_class_size = list_class_size + 1
    for i in range(0,min(20,list_class_size)):
        print(i)
        title_link = list_class[i].find('a', class_='ipc-title-link-wrapper')
        movie_link = "https://www.imdb.com" + title_link['href']
        url_movie = movie_link
        # response = requests.get(url)
        response_movie = requests.get(url_movie, headers=headers)
        soup_movie = BeautifulSoup(response_movie.text,'html.parser')
        # print(soup)# Prettify the HTML
        # pretty_html = soup_movie.prettify()
        title = soup_movie.find('title')
        ratings = soup_movie.find('span',class_='sc-bde20123-1 cMEQkK')
        title_cast = soup_movie.find('section', {'data-testid': 'title-cast'})
        if title_cast:
            title_cast_item = title_cast.find_all('div', {'data-testid':'title-cast-item'})

        directors_writers = soup_movie.find('ul',class_='ipc-metadata-list ipc-metadata-list--dividers-all sc-bfec09a1-8 bHYmJY ipc-metadata-list--base')
        # print(directors_writers)
        if directors_writers:
            dw = directors_writers.find_all('div',class_='ipc-metadata-list-item__content-container')
            l = 0
            for i in dw:
                l =l+1
            directors = []
            writers = []
            if l > 0:
                directors = dw[0].find_all('li',class_='ipc-inline-list__item')
            if l > 1:
                writers = dw[1].find_all('li',class_='ipc-inline-list__item')

        match = re.search(r'tt(\d+)', title_link['href'])
        tt = 'tt' + match.group(1)
        story_link = "https://www.imdb.com/title/" + tt + "/plotsummary/?ref_=tt_ov_pl"
        # print(story_link)
        url_story = story_link
        # response = requests.get(url)
        response_story = requests.get(url_story, headers=headers)
        soup_story = BeautifulSoup(response_story.text,'html.parser')
        # print(soup)# Prettify the HTML
        pretty_html_story = soup_story.prettify()

        story = soup_story.find_all('div',class_="ipc-html-content-inner-div")
        # print(story[1].text)

        user_review = soup_movie.find('section', {'data-testid': 'UserReviews'})
        if user_review:
            user_review_link = user_review.find('a',class_='ipc-title-link-wrapper')
            # print(user_review_link['href'])
            if user_review_link:
                url_reviews = "https://www.imdb.com" + user_review_link['href']
                # response = requests.get(url)
                response_reviews = requests.get(url_reviews, headers=headers)
                soup_reviews = BeautifulSoup(response_reviews.text,'html.parser')
                # print(soup)# Prettify the HTML
                pretty_html_reviews = soup_reviews.prettify()

                reviews = soup_reviews.find('div',class_='lister-list')
                # reviews.prettify()
                rws_title = reviews.find_all('a',class_='title')
                rws_user = reviews.find_all('span',class_='display-name-link')
                user_ratings = reviews.find_all('div',class_='review-container')
                user_ratings_ = []
                for i in user_ratings:
                    j = i.find('span',class_="rating-other-user-rating")
                    if j:
                        s = str(j.text)
                        s_mod = s.replace("\n", "")
                        user_ratings_.append(s_mod)
                    else:
                        user_ratings_.append("NA")
                user_date = reviews.find_all('span',class_='review-date')
                # print()
                user_date_ = []
                for i in user_date:
                    user_date_.append(i.text)
                rws = reviews.find_all('div',class_='text show-more__control')

        entry = {'Title':title.text, 'Ratings': '', 'Cast':'' ,'Directors':'', 'Writers':'', 'Plot Summary': ''}
        if not ratings:
            entry['Ratings'] = '0.0'
        else:
            entry['Ratings'] = ratings.text
        for i in title_cast_item:
            actor = i.find('a', {'data-testid': 'title-cast-item__actor'})
            entry['Cast'] = entry['Cast'] + actor.text + '|'
        entry['Cast'] = entry['Cast'][:-1]
        for i in directors:
            entry['Directors'] = entry['Directors'] + i.text + '|'
        entry['Directors'] = entry['Directors'][:-1]
        for i in writers:
            entry['Writers'] = entry['Writers'] + i.text + '|'
        entry['Writers'] = entry['Writers'][:-1]

        # for j in story:
        #     entry['Plot Summary'] = j.text
        entry['Plot Summary'] = story[0].text
                
        l = 0
        for i in rws:
            l = l+1
        st = "User Review "
        for i in range(0,min(10,l)):
            final_review = rws_title[i].text[:-1] + ': ' + rws[i].text + "~" + rws_user[i].text + " rated " + user_ratings_[i] + " on " + user_date_[i]
            final_review = final_review.replace('\n','')
            # print(final_review)
            entry[st + str(i+1)] = final_review

        if l < 10:
            for i in range(l,10):
                entry[st + str(i+1)] = 'Not Available'

        # print(entry['User Review 10'])
        data.append(entry)

    file_name =  str(genre) +'.csv'
    with open(file_name, 'w', newline='') as csvfile:
        fieldnames = ['Title', 'Ratings', 'Cast', 'Directors','Writers','Plot Summary',"User Review 1","User Review 2", "User Review 3","User Review 4","User Review 5","User Review 6","User Review 7","User Review 8","User Review 9","User Review 10"]
        csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header
        csv_writer.writeheader()

        # Write the data rows
        csv_writer.writerows(data)

In [3]:
url = "https://www.imdb.com/feature/genre#movie"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
# response = requests.get(url)
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text,'html.parser')
# print(soup)# Prettify the HTML
pretty_html = soup.prettify()

movies = soup.find_all('div', class_='ipc-chip-list__scroller')

refs = {}
for i in movies[1]:
    span_element = i.find('span', class_='ipc-chip__text')
    # print(span_element.text)
    refs[span_element.text] = "https://www.imdb.com/" + i['href']

In [4]:
threads = []
for i in refs:
    thread = threading.Thread(target=scrapping_genre,args=(i,refs[i]))
    threads.append(thread)

In [5]:
# Start all threads
start_time = time.time()

for thread in threads:
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

end_time = time.time()

Action
Adventure
Animation
Biography
Comedy
Crime
Documentary
Drama
Family
Fantasy
Film-Noir
History
Horror
Music
Musical
Mystery


Romance
Sci-Fi
Short
Sport
Thriller
War
Western
0
0
0
0
0
0
0
0
00

0
0
0
0
0
0
0
0
0
0
0
0
1
1
11

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
33

3
3
3
3
33
3
3

3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
55
5

5
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
66

6
6
6
6
6
6
6
6
66

6
66

6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
99

9
9
9
9
9
99

9
9
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1011

10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
12
12
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
13
1212

13
13
13
12
13
13
13
13
13
13
13
13
13
14
13
13
13
13
13
1314

14
13
14
14
14
13
13
15
14
14
14
14
15
14
14
14
15
14
15
15
1415

14
14
15
14
1414

14
15
16
15
14
15
16
16
15
15
15
15
16
15
15
16
15
16
16
15
15
17
1515

1616

16
17
16
17
17
16
16
16
16
17
16
16
15
17
17
16
16
16
16
17
17
17
18
17
17
18
17
18
17
17

In [6]:
# scrapping_genre('Documentary',refs['Documentary'])